# Exploration

## Data Cleaning 

#### Summary of what needs to be done to clean the data for each feature

##### Credits dataset

| Feature | Action | Description |
| -------- | ------------|-------------|
| movie_id | Dropped | Dropped during the merging of datasets |
| title | Dropped | Used the title feature from movies dataset |
| cast | Transformed | A list of the cast member names |
| crew | Transformed | A list of the crew member names |


#### Movies dataset
| Feature | Action | Description |
| -------- | ------------|-------------|
| budget | NA | NA |
| genres | Transformed | A list of genres of the film |
| homepage | Dropped! | Not needed for this version of project |
| id | NA | NA |
| keywords | Transformed | A list of keywords of the film |
| original_language | NA | NA |
| original_title |
| overview |
| popularity |
| production_companies |
| production_countries |
| release_date |
| revenue |
| runtime |
| spoken_languages |
| status |
| tagline |
| title |
| vote_average |
| vote_count |

data cleaning reminder -- <strong>Data Quality assessment</strong>

| Category | Completion? | Description |
| -------- | ------------|-------------|
| Accuracy | [ ] | Does it reflect reality over time |
| Completeness | [ ] | is the requisite information there |
| Consistency | [ x ] | Data match with other datasets |
| Currency | [ ] | Values up to date |
| Relevancy | [ ] | Relevant to objective |
| Validity | [ ] | Data containing valuable values |
| No duplicates | [ ] | Only one instance of a data point |